In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time
import re
start_time=time.time()
%matplotlib inline

<h1 align=center> Train Dataset Preparation </h1>

In [2]:
#edo vazoume to proto etsi gia na kseziparoume mono ta train files
import zipfile
archive = zipfile.ZipFile('C:/Users/ilias/Desktop/Project ML/mnistdata.zip')
for file in archive.namelist():
    if file.startswith('train'):
        archive.extract(file,"C:/Users/ilias/Documents")

In [3]:
#edo ginetai to ntaraveri to kalo.
start_time_1=time.time()
rows=[]
train_lengths=[]
for i in range(0,10):
    #mpainoun ta panta grammi grammi apo ola ta files
    with open('train'+str(i)+'.txt') as f:
        rows.append(f.readlines())
    for j in range(0,len(rows[i])):
        #dioxnoume ta \n
        rows[i][j]=rows[i][j][:-1]
        #kratame mono ta integers
        rows[i][j]=[*map(int, rows[i][j].split())]
        #ta kanoume kai np.arrays gia na mas volepsei me to plot
        rows[i][j]=np.array(rows[i][j], dtype='uint8')
    #edo tou apothikeuo poso mikos exei kathe txt diladi poses eikones periexei gia na exoume kai ta sosta labels
    train_lengths.append(len(rows[i]))
#ksediplono tin nested lista gia na exoume se kathe grammi mia eikona        
train_data=[item for sublist in rows for item in sublist]

elapsed_time_1=time.time()-start_time_1 
#ola auta ston mageutiko xrono tou enos leptou kai kati sec
time.strftime("%H:%M:%S", time.gmtime(elapsed_time_1))

'00:01:08'

In [4]:
#edo vlepoume poses eikones apo tin kathe klasi fortosame
train_lengths

[5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]

In [5]:
#edo tsekaroume ean mas efyge kamia eikona (einai ontos 60K)
len(train_data)

60000

In [6]:
def visualization(aux0):
    pixels = aux0.reshape((28, 28))
    plt.imshow(pixels, cmap='gray')
    plt.show()

In [7]:
import itertools
train_labels=[]
for i in range(0,len(train_lengths)):
    train_labels+=list(itertools.repeat(0+i,train_lengths[i]))
#h parakato grammi einai optional ean thelo na kano ta labels strings gia na min mpleksoume me ta noumera pou antiprosopeuoun
#train_labels=list(map(str, train_labels))

#######################
labeled_train_data=pd.Series(train_data,index=[train_labels])
#####################
#auti poia mas dinei oles tis grammes me index 5
labeled_train_data[5]
#eno auti mas dinei tin 5h eikona
labeled_train_data.iloc[5];

<h1 align=center> Test Dataset Preparation </h1>

In [8]:
import zipfile
archive = zipfile.ZipFile('C:/Users/ilias/Desktop/Project ML/mnistdata.zip')
for file in archive.namelist():
    if file.startswith('test'):
        archive.extract(file,"C:/Users/ilias/Documents")

In [9]:
start_time_2=time.time()
rows=[]
test_lengths=[]
for i in range(0,10):
    with open('test'+str(i)+'.txt') as f:
        rows.append(f.readlines())
    for j in range(0,len(rows[i])):
        rows[i][j]=rows[i][j][:-1]
        rows[i][j]=[*map(int, rows[i][j].split())]
        rows[i][j]=np.array(rows[i][j], dtype='uint8')
    test_lengths.append(len(rows[i]))
test_data=[item for sublist in rows for item in sublist]
elapsed_time_2=time.time()-start_time_2 
#ola auta ston mageutiko xrono ton <15 sec
time.strftime("%H:%M:%S", time.gmtime(elapsed_time_2))

'00:00:10'

In [10]:
test_lengths

[980, 1135, 1032, 1010, 982, 892, 958, 1028, 974, 1009]

In [11]:
import itertools
test_labels=[]
for i in range(0,len(test_lengths)):
    test_labels+=list(itertools.repeat(0+i,test_lengths[i]))
labeled_test_data=pd.Series(test_data,index=[test_labels])

<h1 align='center'> SGD </h1>

In [12]:
import sys
import numpy as np
import pandas as pd
import pickle
import gzip
import numpy
import time
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [13]:
# Function for computing the softmax 
def softmax(x):
    
    e = numpy.exp(x - numpy.max(x))  # ensure numerical stability
#     print('entered softmat')
#     print('input of softamax is :\n',x)
#     print('e of softmax is\n',e)
#     print('Sum of es of softmax is,\n')
    if e.ndim == 1:
#         print(numpy.sum(e, axis=0))
        return e / numpy.sum(e, axis=0)
    else:  
#         print(numpy.sum(e, axis=1))
        return e / numpy.array([numpy.sum(e, axis=1)]).T  # ndim = 2

# Create Class: LogisticRegressionClassifier
#Logistic Regression Classifier using Softmax.
# w0 is the bias unit
# w is the weight vector
# x is the feature vector of 1 training sample,
class LogisticRegressionClassifier(object):
    #Initialize the classifier, setting input, labels, W and bias
    def __init__(self, input, labels):
        self.x = input
        self.y = labels
        # initialize W 0
        # random?
        self.W = numpy.zeros((input.shape[1], labels.shape[1])) 
        # initialize bias 0
        self.bias = numpy.zeros(labels.shape[1]) 
        

        
    def train(self,L2_reg ,learning_rate=0.1):
        prob_y_given_x = softmax(numpy.dot(self.x, self.W) + self.bias)
        d_y = self.y - prob_y_given_x
        #print("d_y is",d_y)
        self.W += learning_rate * numpy.dot(self.x.T, d_y) - learning_rate * L2_reg * self.W
        self.bias += learning_rate * numpy.mean(d_y, axis=0)

        
    def negative_log_likelihood(self):
        softmax_activation = softmax(numpy.dot(self.x, self.W) + self.bias)

        result = - numpy.mean(
            numpy.sum(self.y * numpy.log(softmax_activation) +
            (1 - self.y) * numpy.log(1 - softmax_activation), axis=1))
        
        return result


    def predict(self, x):
        return softmax(numpy.dot(x, self.W) + self.bias)        
# END of LogisticRegressionClassifier

In [14]:
#In order to apply the algorithm to the aforementioned datasets and calculate the the percentage of the misclassified
#examples, the following helper functions were implemented:
# Apply Logistic Regression to a Dataset
def apply_lr(x_train, y_train, x_test, L2_regv, learning_rate=0.01, epochs=200):
    # construct LogisticRegression
    # creates class object
    classifier = LogisticRegressionClassifier(input = x_train, labels = y_train)
    # train
    for i in range(epochs):
#         if (i%10==0):
#             print(i)
        classifier.train(L2_reg = L2_regv,learning_rate = learning_rate)
        cost = classifier.negative_log_likelihood()
        # print( '[Training epoch %d] cost: ' % i, cost)
        # learning_rate *= 0.99 #decay learning rate after every epoch
    # predict
    # print( classifier.predict(x_test))
    return classifier.predict(x_test)
    
# Compute the percentage of misclassified examples given the real 
# and the predicted classes
def compute_misclassification_percentage(y,predicted_y):
    if y.shape != predicted_y.shape:
        return( 'Error! Mismatching dimensions of arrays' )    
    y_class = numpy.argmax(y, axis=1)
    predicted_y_class = numpy.argmax(predicted_y, axis=1) 
    percentage = 100 * (y_class != predicted_y_class).sum().astype(float) / y_class.shape[0] 
    print('Misclassifiction percentage: %f%%' % percentage)
    print("Confusion matrix:\n%s" % confusion_matrix(y_class, predicted_y_class))
    print("Classification report: \n%s" % classification_report(y_class, predicted_y_class))
    return percentage

In [15]:
true_labels=list(labeled_train_data.index)
y_train=[np.repeat(0,10)]*len(true_labels)
for i in range(len(y_train)):
    k=true_labels[i]
    a=list(np.repeat(0,k))
    a.append(1)
    y_train[i]=np.array(a+list(np.repeat(0,9-k)))
y_train=np.array(y_train)
################
true_labels_2=test_labels
y_test=[np.repeat(0,10)]*len(true_labels_2)
for i in range(len(y_test)):
    k=true_labels_2[i]
    a=list(np.repeat(0,k))
    a.append(1)
    y_test[i]=np.array(a+list(np.repeat(0,9-k)))
y_test=np.array(y_test)
#################
x_train=np.array(train_data)/255.
#################
x_test=numpy.asarray(test_data)/255.

In [16]:
def apply_lr_to_mnist(x_train, y_train, x_test, y_test, L2_reg , learning_rate=0.00001, epochs=200):

    predictions = apply_lr(x_train, y_train, x_test,L2_reg,
                           learning_rate=learning_rate,
                           epochs=epochs)
             
    return compute_misclassification_percentage(y_test,predictions)         

In [17]:
mnist_lr_misc_perc = apply_lr_to_mnist(x_train,y_train,x_test,y_test, L2_reg=0.0,learning_rate=0.00001, epochs=5)

Misclassifiction percentage: 22.040000%
Confusion matrix:
[[ 940    0    1    3    0    0    8    1   27    0]
 [   0 1038    1    3    0    0    4    1   88    0]
 [  48   37  719   25   18    0   20   18  143    4]
 [  17    5   15  817    1    1    8   15  125    6]
 [   8   12    4    0  774    0   18    1   57  108]
 [  96   19    1  166   17  141   19   18  390   25]
 [  62    7   18    1   11    1  816    0   42    0]
 [  14   47    7    0   11    0    1  855   55   38]
 [  13    8    2   27    6    0    4   10  898    6]
 [  28   12    7    8   48    0    1   35   72  798]]
Classification report: 
             precision    recall  f1-score   support

          0       0.77      0.96      0.85       980
          1       0.88      0.91      0.89      1135
          2       0.93      0.70      0.80      1032
          3       0.78      0.81      0.79      1010
          4       0.87      0.79      0.83       982
          5       0.99      0.16      0.27       892
          6    

# Validation - Regularization Parameter


# Neural net 

In [18]:
def activation_f(x):
    import numpy as np
    output=np.log(1+np.exp(x))
    return output

In [19]:
def activation2_f(x):
    import numpy as np
    output=(np.exp(x)-np.exp(np.negative(x)))/(np.exp(x)+np.exp(np.negative(x)))
    return output

In [20]:
def softmax2(w):
    w = np.array(w)
    maxes = np.amax(w, axis=1)
    maxes = maxes.reshape(maxes.shape[0], 1)
    e = np.exp(w - maxes)
    dist = e / np.sum(e, axis=1).reshape(maxes.shape[0], 1)
    return dist

In [21]:
def gradient_softmax(x):
    import numpy as np
    x=softmax2(x)
    gradient=x.copy()
    for i in range(len(x)):
        for j in range(len(x)):
            if i == j:
                gradient[i] = x[i] * (1-x[i])
            else: 
                gradient[i] = -x[i]*x[j]
    return gradient

In [22]:
y_train=labeled_train_data.index
X=np.asarray(train_data)
y=np.zeros(60000)
for i in range(len(y_train)):
    y[i]=np.asarray(y_train[i],dtype=int)
y=y.reshape(-1,1)    

y_onehot=y_train
X=np.asarray(train_data)

In [23]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(60000, 10)

In [24]:
input_size = 784
hidden_size = 3
num_labels = 10
regularization_parameter = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((3, 785), (10, 4))

In [25]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(activation_f(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = softmax2(z3)
    
    return a1, z2, a2, z3, h

In [26]:
def cross_entropy(params, input_size, hidden_size, num_labels, X, y, regularization_parameter):
    m = X.shape[0]
    n=X.shape[1]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0.0
    for i in range(m):
        for j in range(10):
            aux = np.multiply(y[i,j], np.log(h[i,j]))
            J += aux
    J=J/m
    
    # add the cost regularization term
    J -= (float(regularization_parameter) / 2) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [27]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((60000, 785), (60000, 3), (60000, 4), (60000, 10), (60000, 10))

In [28]:
cross_entropy(params, input_size, hidden_size, num_labels, X, y_onehot, regularization_parameter)

-25.259446391554494

In [29]:
def backprop1(params, input_size, hidden_size, num_labels, X, y, regularization_parameter):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
   
    J = cross_entropy(params, input_size, hidden_size, num_labels, X, y_onehot, regularization_parameter)
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, gradient_softmax(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
     # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * regularization_parameter) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * regularization_parameter) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [30]:
J, grad = backprop1(params, input_size, hidden_size, num_labels, X, y_onehot, regularization_parameter)
J, grad.shape

(-25.259446391554494, (2395,))

# Gradient Check

# Optimizer

In [31]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop1, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, regularization_parameter), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

KeyboardInterrupt: 

In [32]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

NameError: name 'fmin' is not defined

In [ ]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
#print 'accuracy = {0}%'.format(accuracy * 100)
accuracy